# Correlation Assignment

If you are using Google Colab to finish your assignment, make a copy to your own drive and edit.
If you are running on your own local machine, you can install the `colab` extension in your own VSCode, and running with the colab kernel.


**Your name (unique name)**: npisar

**Your U-M ID**: 07607086

In [18]:
# Load libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Helper Functions

To get p-values, you have to use a library specifically built for statistical testing, like _SciPy_.

`scipy` is the gold standard for scientific computing, but it is designed to calculate the correlation and p-value for one pair of variables at a time. It doesn't have a single "matrix" function like `df.corr()`.

To create the correlation matrix and get p-values using `scipy`, you need to iterate through your column combinations.

In [19]:
def get_scipy_corrs(df, variables):
    """
    This is a helper function that returns a correlation matrix with p-values from scipy.

    Args:
      df (pandas.DataFrame): The input DataFrame.
      variables (list): A list of column names to calculate correlations for.

    Returns:
      tuple: A tuple containing two pandas DataFrames:
        - r_df (pandas.DataFrame): Correlation matrix with correlation coefficients.
        - p_df (pandas.DataFrame): Correlation matrix with p-values.
    """
    n = len(variables)
    # Initialize with NaNs to handle cases where calculation fails
    p_matrix = np.full((n, n), np.nan)
    r_matrix = np.full((n, n), np.nan)

    for i in range(n):
        for j in range(n):
            # 1. Select the two columns and drop rows where either is NaN
            subset = df[[variables[i], variables[j]]].dropna()

            # 2. Ensure data is numeric and flattened to 1D arrays
            x = subset[variables[i]].astype(float).values
            y = subset[variables[j]].astype(float).values

            # 3. Calculate (requires at least 2 points for a correlation)
            if len(x) > 1:
                try:
                    res = stats.pearsonr(x, y)
                    r_matrix[i, j] = res.statistic
                    p_matrix[i, j] = res.pvalue
                except:
                    continue # Skip pairs that cause math errors (e.g. zero variance)

    # Return as clean DataFrames
    r_df = pd.DataFrame(r_matrix, index=variables, columns=variables)
    p_df = pd.DataFrame(p_matrix, index=variables, columns=variables)
    return r_df, p_df


def format_sig_stars(p):
    if p < 0.001: return '***'
    if p < 0.01: return '**'
    if p < 0.05: return '*'
    return ''

def combine_r_and_p(r_df, p_df):
    """
    This is a helper function that combines the correlation and p-values.
    """
    combined = r_df.copy().astype(str)

    for i in range(len(r_df.index)):
        for j in range(len(r_df.columns)):
            r_val = r_df.iloc[i, j]
            p_val = p_df.iloc[i, j]

            if pd.isna(r_val) or pd.isna(p_val):
                combined.iloc[i, j] = ""
                continue

            stars = format_sig_stars(p_val)
            # Format: 0.123*** (p=0.002)
            combined.iloc[i, j] = f"{r_val:.3f}{stars}\n({p_val:.3f})"

    return combined


Creating a table that follows APA conventions is a little tricky. We've written a function to help you.

In [20]:
def export_apa_correlation(r_df, p_df):
    # Create a mask for the upper triangle
    mask = np.triu(np.ones_like(r_df, dtype=bool))

    # Initialize the formatted DataFrame
    apa_df = r_df.copy().astype(str)

    for i in range(len(r_df.index)):
        for j in range(len(r_df.columns)):
            # Hide the upper triangle and diagonal
            if i <= j:
                apa_df.iloc[i, j] = ""
                continue

            r_val = r_df.iloc[i, j]
            p_val = p_df.iloc[i, j]

            # Add stars
            stars = ""
            if p_val < .001: stars = "***"
            elif p_val < .01: stars = "**"
            elif p_val < .05: stars = "*"

            # Format to 2 or 3 decimal places (APA usually uses 2 or 3)
            # This version keeps 3 for precision
            formatted_r = f"{r_val:.2f}".replace("0.", ".") # Remove leading zero for APA style
            apa_df.iloc[i, j] = f"{formatted_r}{stars}"

    # Rename columns to 1, 2, 3... to follow APA table headers
    apa_df.columns = [f"{i+1}." for i in range(len(apa_df.columns))]
    # Add the variable names as the first column
    apa_df.insert(0, "Variable", r_df.index)
    # Add a column for the index numbers
    apa_df.insert(0, "№", range(1, len(apa_df) + 1))

    return apa_df

## Load the data

You'll need to download the data from the link in the assignment on Canvas. The filename you're looking for is `Dawtry Sutton and Sibley 2015 Study 1a.csv`.

In [21]:
"""
This is some extra stuff for google colab, don't need it
########################################################

# # Load the Dawtry et al. (2015) Study 1a data
# file_id = '0Bz-rhZ21ShvOMGxnYUJfYmR5d2M'
# resource_key = '0-jo7UtjyXsahMUKXVOQvb9g'

# # Construct a direct download link
# direct_link = f'https://drive.google.com/uc?export=download&id={file_id}&resourcekey={resource_key}'
# df = pd.read_csv(direct_link)
"""

# Set directories
# MAC
data_dir = '/Users/nick/Desktop/School/Winter 2026/SI 313/SI313-homework/data'
# PC
# data_dir = 'F:/School/College/WN 2026/SI 313/SI313-WN26-personal/data'

# change the directory to the data
os.chdir(data_dir)

# Load the Dawtry et al. (2015) Study 1a data
df = pd.read_csv('correlation-assignment_Dawtry Sutton and Sibley 2015 Study 1a.csv')

# Explore the dataframe
print(f"Shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")
print(f"\nFirst few rows:\n{df.head()}")

Shape: (305, 37)

Column names:
['PS', 'PD_15', 'PD_30', 'PD_45', 'PD_60', 'PD_75', 'PD_90', 'PD_105', 'PD_120', 'PD_135', 'PD_150', 'PD_150plus', 'fairness', 'satisfaction', 'SC_15', 'SC_30', 'SC_45', 'SC_60', 'SC_75', 'SC_90', 'SC_105', 'SC_120', 'SC_135', 'SC_150', 'SC_150plus', 'redist1', 'redist2', 'redist3', 'redist4', 'Household_Income', 'Political_Preference', 'age', 'gender', 'Population_Inequality_Gini_Index', 'Population_Mean_Income', 'Social_Circle_Inequality_Gini_Index', 'Social_Circle_Mean_Income']

First few rows:
    PS  PD_15  PD_30  PD_45  PD_60  PD_75  PD_90  PD_105  PD_120  PD_135  ...  \
0  233     27     48     21      0      0      0       0       0       0  ...   
1  157     39      0      0      0      0      0       0       0       0  ...   
2  275      0      0     50      0      0     50       0       0       0  ...   
3  111      9     14     17     17     17      8       7       5       2  ...   
4   52     68     32      0      0      0      0       0    

## Data cleaning

This section TO-DO list:
  1. Understand the data with descriptive analysis
  2. Compute two new variables

Compute the two variables:  
1. You should first create a score that captures participants’ perceptions that the current system is fair and satisfactory. To do this, COMPUTE the mean for the items fairness and satisfaction, naming this new variable `fairness_and_satisfaction`.    
2. Next, you should create a score that captures participants’ support for redistribution. The researchers asked participants four questions in total, two asked about their support for redistribution, and two asked about their opposition to redistribution.
3. To create a single score that reflect participants overall view toward redistribution, we first need to recode the two items that assess opposition to redistribution. Reverse score redist2 and redist4, so that 6 = 1, 5 = 2, 4 = 3, 3 = 4, 2 = 5, 1 = 6. Name the recoded variables `redist2_recode` and `redist4_recode`.
4. Now, COMPUTE the mean for the items `redist1`, `redist2_recode`, `redist3`, `redist4_recode`, naming this new variable `support_for_redistribution`.


In [22]:
# These are the relevant variables we'll use
relevant_vars = ['fairness', 'satisfaction',
                  'redist1', 'redist2', 'redist3', 'redist4',
                  'Household_Income', 'Social_Circle_Mean_Income',
                  'Population_Mean_Income', 'Social_Circle_Inequality_Gini_Index',
                  'Population_Inequality_Gini_Index', 'Political_Preference']

print(f"\nRelevant Variables")
print(f"{"="*50}")
print(f"{relevant_vars}")


Relevant Variables
['fairness', 'satisfaction', 'redist1', 'redist2', 'redist3', 'redist4', 'Household_Income', 'Social_Circle_Mean_Income', 'Population_Mean_Income', 'Social_Circle_Inequality_Gini_Index', 'Population_Inequality_Gini_Index', 'Political_Preference']


In [23]:
# Data description, you can use .info() or .describe()
## TODO: Conduct descriptive analysis, notice any missingness and weird stats
## Remember to check the variables relevant to our study

# .info() to check the missingness
# Look at data types and missing values
print(f"Data Types")
print(f"{"="*50}")
print(df.info())

# .describe() to easily check the descriptive stats of the dataset
# Get basic stats for all numeric variables
print(f"\n\n\n\n\nBasic Stats")
print(f"{"="*50}")
print(df.describe())


# print the descriptive stats of the relevant variables 
print(f"\n\n\n\n\nRelevant Variables")
print(f"{"="*50}")
print(df[relevant_vars].describe())

Data Types
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   PS                                   305 non-null    int64  
 1   PD_15                                305 non-null    int64  
 2   PD_30                                305 non-null    int64  
 3   PD_45                                305 non-null    int64  
 4   PD_60                                305 non-null    int64  
 5   PD_75                                305 non-null    int64  
 6   PD_90                                305 non-null    int64  
 7   PD_105                               305 non-null    int64  
 8   PD_120                               305 non-null    int64  
 9   PD_135                               305 non-null    int64  
 10  PD_150                               305 non-null    int64  
 11  PD_150plus           

In [24]:
# Creating two new variables
## First step: compute the mean of fairness and satisfaction
df['fairness_and_satisfaction'] = df[['fairness', 'satisfaction']].mean(axis=1)

## Remember to double check if the new variable works
print(f"New variable created: {df['fairness_and_satisfaction'].describe()}")
print(f"\nMissing values: {df['fairness_and_satisfaction'].isna().sum()}")

# Check a few cases manually
print(df[['fairness', 'satisfaction', 'fairness_and_satisfaction']].head(10)) # use this code to check

New variable created: count    305.000000
mean       3.539344
std        2.024950
min        1.000000
25%        2.000000
50%        3.000000
75%        5.000000
max        9.000000
Name: fairness_and_satisfaction, dtype: float64

Missing values: 0
   fairness  satisfaction  fairness_and_satisfaction
0         1             1                        1.0
1         5             2                        3.5
2         5             5                        5.0
3         7             7                        7.0
4         4             5                        4.5
5         1             4                        2.5
6         3             3                        3.0
7         5             4                        4.5
8         5             3                        4.0
9         4             5                        4.5


In [25]:
## Second step: reverse the redist2 and redist 4, you can use the formula: reversed = 7 - original
df['redist2_recode'] = 7 - df['redist2']
df['redist4_recode'] = 7 - df['redist4']

# Verify a few cases like what we did for fairness and satisfaction
print("Original redist2 values:")
print(f"{"="*50}")
print(df['redist2'].value_counts().sort_index())

print("\n\n\nRecoded redist2 values:")
print(f"{"="*50}")
print(df['redist2_recode'].value_counts().sort_index())

# Check a few cases
print("\n\n\nSample of recoding:")
print(f"{"="*50}")
print(df[['redist2', 'redist2_recode', 'redist4', 'redist2_recode']].head(10))

Original redist2 values:
redist2
1    78
2    97
3    77
4    31
5    15
6     7
Name: count, dtype: int64



Recoded redist2 values:
redist2_recode
1     7
2    15
3    31
4    77
5    97
6    78
Name: count, dtype: int64



Sample of recoding:
   redist2  redist2_recode  redist4  redist2_recode
0        3               4        1               4
1        2               5        4               5
2        4               3        5               3
3        3               4        4               4
4        5               2        5               2
5        5               2        6               2
6        2               5        5               5
7        3               4        4               4
8        4               3        5               3
9        4               3        5               3


In [30]:
## Third step: compute the mean of all redistribution items
df['support_for_redistribution'] = df[['redist1', 'redist2_recode', 'redist3', 'redist4_recode']].mean(axis=1)

# Verify again
print(f"New variable created:\n{"="*50}\n{df['support_for_redistribution'].describe()}")
print(f"\n\n\nMissing values:\n{"="*50}\n{df['support_for_redistribution'].isna().sum()}")

# Check internal consistency (Cronbach's alpha - optional)
redist_items = df[['redist1', 'redist2_recode', 'redist3', 'redist4_recode']]
print(redist_items.corr())

New variable created:
count    305.000000
mean       3.909836
std        1.149436
min        1.000000
25%        3.250000
50%        4.000000
75%        4.750000
max        6.000000
Name: support_for_redistribution, dtype: float64



Missing values:
0
                 redist1  redist2_recode   redist3  redist4_recode
redist1         1.000000        0.360760  0.754321        0.512747
redist2_recode  0.360760        1.000000  0.442748        0.446623
redist3         0.754321        0.442748  1.000000        0.509098
redist4_recode  0.512747        0.446623  0.509098        1.000000


After we have done many cleaning for our dataset, it is important to check the descriptives again **before** doing any analyses.

In [32]:
# List of relevant variables we are going to use
key_vars = ['Household_Income', 'Social_Circle_Mean_Income',
                  'Population_Mean_Income', 'Social_Circle_Inequality_Gini_Index',
                  'Population_Inequality_Gini_Index', 'fairness_and_satisfaction',
            'support_for_redistribution', 'Political_Preference']
# debug
# print(df.columns)

# Check their descriptive analysis
descriptives = df[key_vars].describe().loc[['mean', 'std']].T
descriptives.columns = ['M', 'SD']
descriptives = descriptives. round(2)

print("Descriptive Statistics for Key Variables")
print("="*50)
print (descriptives)

# Also get sample sizes
print(f"\n\n\nSample Size per variable")
print("="*50)
print(df[key_vars].count())

Descriptive Statistics for Key Variables
                                            M        SD
Social_Circle_Mean_Income            54294.00  25295.52
Population_Mean_Income               58604.85  17230.52
Social_Circle_Inequality_Gini_Index     26.35      9.97
Population_Inequality_Gini_Index        35.51      7.48
fairness_and_satisfaction                3.54      2.02
support_for_redistribution               3.91      1.15



Sample Size per variable
Household_Income                       305
Social_Circle_Mean_Income              305
Population_Mean_Income                 305
Social_Circle_Inequality_Gini_Index    305
Population_Inequality_Gini_Index       305
fairness_and_satisfaction              305
support_for_redistribution             305
Political_Preference                   305
dtype: int64


In [28]:
# Or we can present it in a formatted way:
# Calculate descriptive statistics
descriptives = df[key_vars].describe().loc[['mean', 'std', 'count']].T
descriptives.columns = ['M', 'SD', 'N']
descriptives = descriptives.round(2)

print("Descriptive Statistics for Key Variables")
print("="*50)
print(descriptives)

Descriptive Statistics for Key Variables
                                            M        SD      N
Social_Circle_Mean_Income            54294.00  25295.52  305.0
Population_Mean_Income               58604.85  17230.52  305.0
Social_Circle_Inequality_Gini_Index     26.35      9.97  305.0
Population_Inequality_Gini_Index        35.51      7.48  305.0
fairness_and_satisfaction                3.54      2.02  305.0
support_for_redistribution               3.91      1.15  305.0


Notice anything weird? If not, we can proceed!

## Correlation Analysis

We are going through a typical workflow of conducting correlation analysis in Python
- Check the data type `dtype` of variables: ensure all variables in correlation analysis are numeric, you can use `pd.to_numeric()` to transform non-numeric variables to numeric
- Create a Pearson correlation matrix using `df.corr(method='pearson')` and try to visualize it
- Use the helper function `get_scipy_corrs()` to get a correlation matrix with p-values
- Finally, present our results in APA format using `export_apa_correlation()`

Now we first start from checking data types. You can use `.dtypes` in Pandas to check the data types.

In [37]:
# Checking data types
print(df[key_vars].dtypes)

Household_Income                       float64
Social_Circle_Mean_Income                int64
Population_Mean_Income                   int64
Social_Circle_Inequality_Gini_Index    float64
Population_Inequality_Gini_Index       float64
fairness_and_satisfaction              float64
support_for_redistribution             float64
Political_Preference                   float64
dtype: object


Do you notice any variables that are not numeric? Try to transform them.

In [38]:
# Transform the non-numeric variables using pd.numeric()
df[key_vars] = df[key_vars].apply(pd.to_numeric, errors='coerce')

print(f"\nCheck Updated Types")
print(f"{"="*50}")
print(df[key_vars].dtypes)


Check Updated Types
Household_Income                       float64
Social_Circle_Mean_Income                int64
Population_Mean_Income                   int64
Social_Circle_Inequality_Gini_Index    float64
Population_Inequality_Gini_Index       float64
fairness_and_satisfaction              float64
support_for_redistribution             float64
Political_Preference                   float64
dtype: object


Then, create a correlation matrix and a heatmap

In [39]:
# Calculate corrleation matrix
# TODO: write the code here
correlation_matrix = df[key_vars].corr(method='pearson')
r_values = correlation_matrix


# Present it in a formatted way
print("Correlation matrix")
print("="*50)
print(correlation_matrix.round(2))

Correlation matrix
                                     Household_Income  \
Household_Income                                 1.00   
Social_Circle_Mean_Income                        0.48   
Population_Mean_Income                           0.19   
Social_Circle_Inequality_Gini_Index             -0.12   
Population_Inequality_Gini_Index                -0.07   
fairness_and_satisfaction                        0.18   
support_for_redistribution                      -0.21   
Political_Preference                             0.15   

                                     Social_Circle_Mean_Income  \
Household_Income                                          0.48   
Social_Circle_Mean_Income                                 1.00   
Population_Mean_Income                                    0.34   
Social_Circle_Inequality_Gini_Index                      -0.10   
Population_Inequality_Gini_Index                         -0.15   
fairness_and_satisfaction                                 0.24   
suppo

**Interpretation**: Explain the correlation matrix briefly. Tell us about what you found from the matrix in 2-3 sentences.
(Respond here)

**Response**


To get p-values, you have to use a library specifically built for statistical testing, like _SciPy_.

`scipy` is the gold standard for scientific computing, but it is designed to calculate the correlation and p-value for one pair of variables at a time. It doesn't have a single "matrix" function like `df.corr()`.

To create the correlation matrix and get p-values using `scipy`, you need to iterate through your column combinations. Here we are using the helper function `get_scipy_corrs(df, variables)`. **In your own project, you need write your own helper functions.**

In [ ]:
# Get correlation matrix with p-values
...

# Generate the report with p-values and significance stars
report_df = combine_r_and_p(...)
print(report_df)

Prepare a correlation matrix that includes all of the relevant study variables. Make sure to follow APA-style guidelines.

Tips: You can use `export_apa_correlation()`

In [ ]:
apa_table = ...
apa_table

**BONUS**: draw a heatmap of the correlation matrix. Sometimes tables are overwhelming. We can use `seaborn` to draw a heatmap visualization and color the cells by the correlation strength.

Tips: You can make a good use of the `heatmap()` in seaborn

In [ ]:
# Draw the heatmap here: